In [ ]:
from sklearn import metrics

In [ ]:
#LSTM models
# input shape format is [samples, timesteps, features]
# base has [30, 1, 9]

In [ ]:
def format_to_lstm(df):
    X = np.array(df)
    return np.reshape(X, (X.shape[0], 1, X.shape[1]))

In [ ]:
# Transform the datasets for LSTM processing
lstm_X_test1_summaryL1Z2_std = format_to_lstm(X_test1_summaryL1Z2_std)
lstm_X_test1_summaryL1Z2_minMax = format_to_lstm(X_test1_summaryL1Z2_minMax)
lstm_X_train_base_std = format_to_lstm(X_train_base_std)
lstm_X_train_base_minMax = format_to_lstm(X_train_base_minMax)
lstm_X_train_all_std = format_to_lstm(X_train_all_std)
lstm_X_train_all_minMax = format_to_lstm(X_train_all_minMax)
lstm_X_test_all_std = format_to_lstm(X_test_all_std)
lstm_X_test_all_minMax = format_to_lstm(X_test_all_minMax)

In [ ]:
# Neural Nets imports
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten 
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, BatchNormalization
from tensorflow.keras.optimizers import Adam, Adadelta, Adagrad, Adamax, SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.regularizers import l1, l2, l1_l2

In [ ]:
from tensorflow.keras.utils import multi_gpu_model

In [ ]:
#Define LSTM model
#input shape for the LSTM is (number_steps x number_features)

def lstm_model_mae(layerSize, numSteps, numFeatures):
    
    model = Sequential()
    model.add(LSTM(layerSize, return_sequences=True, input_shape=(numSteps,numFeatures),
                    #kernel_regularizer=l1(0.01), bias_regularizer=l1(0.01),
                    #kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),
                    #kernel_regularizer=l1_l2(0.01), bias_regularizer=l1_l2(0.01),                   
                   dropout=0.2, recurrent_dropout=0.2))
    model.add(LSTM(layerSize, input_shape=(numSteps,numFeatures), 
                    #kernel_regularizer=l1(0.01), bias_regularizer=l1(0.01),
                    #kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),
                    #kernel_regularizer=l1_l2(0.01), bias_regularizer=l1_l2(0.01),
                   dropout=0.2, recurrent_dropout=0.2))

    model.add(Dense(layerSize, kernel_initializer='normal',
                    #kernel_regularizer=l1(0.01), bias_regularizer=l1(0.01),
                    #kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),
                    #kernel_regularizer=l1_l2(0.01), bias_regularizer=l1_l2(0.01),
                    activation=custom_activation))
    #model.add(BatchNormalization())
    
    model.add(Dense(1, kernel_initializer='normal', 
                    #kernel_regularizer=l1(0.01), bias_regularizer=l1(0.01),
                    #kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),
                    #kernel_regularizer=l1_l2(0.01), bias_regularizer=l1_l2(0.01),
                    activation='linear'))
    
    model.compile(Adam(lr=0.001),
              loss='mae',
              metrics=['mae'])
    return model

In [ ]:
verboseLevel=0
validationSplit=0.2
batchSize=30
epochs=1000

In [ ]:
# callback preparation
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.5,
                              patience=2,
                              verbose=verboseLevel,
                              mode='min',
                              min_lr=0.001)

In [ ]:
layerSize = 64
numSteps = 1
numFeatures = 9

In [ ]:
# STD ALL

In [ ]:
checkpoint_lstm_std_all_mae = ModelCheckpoint("lstm_model_std_all_mae.h5",
                             monitor='val_loss',
                             verbose=verboseLevel,
                             save_best_only=True,
                             mode='min')

callbacks_list_lstm_std_all_mae = [checkpoint_lstm_std_all_mae, reduce_lr]

In [ ]:
lstm_std_all_mae_model = lstm_model_mae(layerSize=layerSize, numSteps=numSteps, numFeatures=numFeatures)

In [ ]:
%%time
history_LSTM_std_all_mae = lstm_std_all_mae_model.fit(lstm_X_train_all_std, y_train_scaled_std_all,
                                batch_size=batchSize, 
                                validation_split=validationSplit, 
                                epochs=epochs, verbose=verboseLevel,
                                callbacks=callbacks_list_lstm_std_all_mae)

In [ ]:
##################################################
# LSTM STD ALL MAE
#################################################
#Reload the best saved model
lstm_std_all_mae_model_new = load_model('lstm_model_std_all_mae.h5')
#Predict
y_pred_LSTM_std_all_mae = lstm_std_all_mae_model_new.predict(lstm_X_test_all_std)
pred = targetStdScalerAll.inverse_transform(y_pred_LSTM_std_all_mae)

In [ ]:
minMseAWS = metrics.mean_squared_error(y_test_all_std, pred)
minRmseAWS = np.sqrt(minMseAWS)
minMaeAWS = metrics.mean_absolute_error(y_test_all_std, pred)
minR2AWS = metrics.r2_score(y_test_all_std, pred)
modelNameAWS = "LSTM_STD_ALL_MAE"

In [ ]:
delta = pd.DataFrame()
delta = (1-(pd.DataFrame(pred)  / y_test_all_std.to_numpy()))*100
delta = delta.rename(columns={delta.columns[0]: "ThroughputDeltaDeviationPercentage"})
#delta

In [ ]:
mpd_all = delta.median()[0]

In [ ]:
def mape(test, pred):
    x = 0
    for i in range(len(test)):
        x+=np.abs((test[test.columns[0]][i] - pred[i])/test[test.columns[0]][i])
    return (x/len(test)*100)

In [ ]:
mape_all = mape(y_test_all_std, pred)

In [ ]:
minMaeAWS, minR2AWS, modelNameAWS, minMseAWS, minRmseAWS

In [ ]:
#### END ####